In [ ]:
%pip install azure-storage-blob
%pip install pandas pyarrow
%pip install newsapi-python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for newsapi-python from https://files.pythonhosted.org/packages/74/47/e3b099102f0c826d37841d2266e19f1568dcf58ba86e4c6948e2a124f91d/newsapi_python-0.2.7-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
dbutils.library.restartPython()

### Import necessary Dependencies

In [1]:
import pandas as pd
import os
import io
from azure.storage.blob import BlobServiceClient, BlobClient
from newsapi import NewsApiClient
from pandas import json_normalize
import json
import csv

### Extraction Layer

In [2]:
# Init
api = NewsApiClient(api_key='0db6fa273f154cbc9409997bd556a55b')
keywords_list = ['commodities', 'agriculture', 'metals', 'energy']

all_articles_list = []

for keyword in keywords_list:
    articles =api.get_everything(q=keyword)
    all_articles_list.extend(articles['articles'])

with open('articles.json', 'w') as file:
    json.dump(all_articles_list, file, indent=4)


In [3]:
nunu_df = pd.read_json('/Workspace/Users/ihekakelechi@outlook.com/articles.json')

In [4]:
nunu_df.columns

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')

In [5]:
nunu_df.head(3)

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Archie Hunter,"After Rough Patch, Trader Engelhart Bets Big o...",(Bloomberg) -- The last time Engelhart Commodi...,https://finance.yahoo.com/news/rough-patch-tra...,https://s.yimg.com/ny/api/res/1.2/logpAHtvT.V5...,2024-07-04T11:57:36Z,(Bloomberg) -- The last time Engelhart Commodi...
1,"{'id': 'business-insider', 'name': 'Business I...",mfox@businessinsider.com (Matthew Fox),Stock market today: Dow closes at a record as ...,Investors betting on a second Trump presidency...,https://markets.businessinsider.com/news/stock...,https://i.insider.com/669580569caa6ed01e01c687...,2024-07-15T20:09:18Z,"US stocks hit new all-time highs on Monday, wi..."
2,"{'id': 'business-insider', 'name': 'Business I...",mfox@businessinsider.com (Matthew Fox),Stock market today: Indexes test record highs ...,"The June jobs ""should have Fed officials troub...",https://markets.businessinsider.com/news/stock...,https://i.insider.com/6687f1d71f0c1cd6d731bbbf...,2024-07-05T13:39:38Z,US stocks tested record highs on Friday after ...


In [6]:
source = json_normalize(nunu_df['source'])

In [ ]:
source.head(5)

,id,name
0,None,Yahoo Entertainment
1,wired,Wired
2,None,Yahoo Entertainment
3,business-insider,Business Insider
4,business-insider,Business Insider


In [7]:
nunu_df = nunu_df.drop(['source'], axis=1).join(source)

In [ ]:
nunu_df.head(3)

,author,title,description,url,urlToImage,publishedAt,content,id,name
0,Archie Hunter,"After Rough Patch, Trader Engelhart Bets Big o...",(Bloomberg) -- The last time Engelhart Commodi...,https://finance.yahoo.com/news/rough-patch-tra...,https://s.yimg.com/ny/api/res/1.2/logpAHtvT.V5...,2024-07-04T11:57:36Z,(Bloomberg) -- The last time Engelhart Commodi...,None,Yahoo Entertainment
1,Nitish Pahwa,Everything’s About to Get a Hell of a Lot More...,"Intensifying hurricanes, floods, and heat wave...",https://www.wired.com/story/everythings-about-...,https://media.wired.com/photos/6672c1cb766de1f...,2024-06-22T11:00:00Z,Agricultural yields for important commodities ...,wired,Wired
2,Reuters,Minimum price mechanism for Ukrainian food exp...,New rules to prevent Ukranian agricultural exp...,https://www.yahoo.com/news/minimum-price-mecha...,https://media.zenfs.com/en/reuters.com/2b6a7d2...,2024-06-20T07:39:52Z,KYIV (Reuters) - New rules to prevent Ukranian...,None,Yahoo Entertainment


In [ ]:
nunu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   author       344 non-null    object
 1   title        400 non-null    object
 2   description  384 non-null    object
 3   url          400 non-null    object
 4   urlToImage   377 non-null    object
 5   publishedAt  400 non-null    object
 6   content      400 non-null    object
 7   id           85 non-null     object
 8   name         400 non-null    object
dtypes: object(9)
memory usage: 28.2+ KB


In [8]:
nunu_df= nunu_df[['id', 'name','author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', \
                   'content']]

In [9]:
nunu_df.columns

Index(['id', 'name', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')

In [10]:
author_r = nunu_df[['author','name']].copy().drop_duplicates().reset_index(drop=True)
author_r = author_r.reset_index().rename(columns={'index':'author_id'})

In [11]:
author_r.head(5)

,author_id,author,name
0,0,Archie Hunter,Yahoo Entertainment
1,1,mfox@businessinsider.com (Matthew Fox),Business Insider
2,2,fdemott@insider.com (Filip De Mott),Business Insider
3,3,Calculated Risk,Blogger.com
4,4,Zacks Equity Research,Yahoo Entertainment


In [13]:
source_r = nunu_df[['url', 'urlToImage']].copy().drop_duplicates()
source_r = source_r.reset_index().rename(columns={'index':'source_id'})

In [14]:
source_r.head(5)

,source_id,url,urlToImage
0,0,https://finance.yahoo.com/news/rough-patch-tra...,https://s.yimg.com/ny/api/res/1.2/logpAHtvT.V5...
1,1,https://markets.businessinsider.com/news/stock...,https://i.insider.com/669580569caa6ed01e01c687...
2,2,https://markets.businessinsider.com/news/stock...,https://i.insider.com/6687f1d71f0c1cd6d731bbbf...
3,3,https://markets.businessinsider.com/news/stock...,https://i.insider.com/669676e45439fb59cebdb3ea...
4,4,https://www.blogger.com/comment.g?blogID=10004...,None


In [15]:
article_r =nunu_df.merge(author_r, on=['author','name'], how='left') \
                .merge(source_r, on=['url','urlToImage'], how='left')
article_r=article_r.reset_index().rename(columns={'index':'article_id'})
article_r = article_r[['article_id','author_id','source_id','id', 'title', 'description','publishedAt', 'content']]

article_r.head(3)            

,article_id,author_id,source_id,id,title,description,publishedAt,content
0,0,0,0,None,"After Rough Patch, Trader Engelhart Bets Big o...",(Bloomberg) -- The last time Engelhart Commodi...,2024-07-04T11:57:36Z,(Bloomberg) -- The last time Engelhart Commodi...
1,1,1,1,business-insider,Stock market today: Dow closes at a record as ...,Investors betting on a second Trump presidency...,2024-07-15T20:09:18Z,"US stocks hit new all-time highs on Monday, wi..."
2,2,1,2,business-insider,Stock market today: Indexes test record highs ...,"The June jobs ""should have Fed officials troub...",2024-07-05T13:39:38Z,US stocks tested record highs on Friday after ...


### Transformation Layer

In [16]:
author_r.info()
source_r.info()
article_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  260 non-null    int64 
 1   author     250 non-null    object
 2   name       260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source_id   391 non-null    int64 
 1   url         391 non-null    object
 2   urlToImage  378 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   article_id   400 non-null    int64 
 1   author_id    400 non-null    int64 
 2   source_id    400 non-nul

In [17]:
author = author_r.fillna('unknown')

In [18]:
author.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  260 non-null    int64 
 1   author     260 non-null    object
 2   name       260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.2+ KB


In [19]:
source = source_r.fillna('unknown')

In [20]:
source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source_id   391 non-null    int64 
 1   url         391 non-null    object
 2   urlToImage  391 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.3+ KB


In [22]:
article = article_r.fillna({
    'id':'unknown',
    'description':'unknown'})

In [23]:
article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   article_id   400 non-null    int64 
 1   author_id    400 non-null    int64 
 2   source_id    400 non-null    int64 
 3   id           400 non-null    object
 4   title        400 non-null    object
 5   description  400 non-null    object
 6   publishedAt  400 non-null    object
 7   content      400 non-null    object
dtypes: int64(3), object(5)
memory usage: 25.1+ KB


In [24]:
article['publishedAt'] = pd.to_datetime(article['publishedAt'])

In [25]:
article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   article_id   400 non-null    int64              
 1   author_id    400 non-null    int64              
 2   source_id    400 non-null    int64              
 3   id           400 non-null    object             
 4   title        400 non-null    object             
 5   description  400 non-null    object             
 6   publishedAt  400 non-null    datetime64[ns, UTC]
 7   content      400 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(4)
memory usage: 25.1+ KB


In [ ]:
# Create the directory
dbutils.fs.mkdirs("/dbfs/dataset/rawdata")


In [ ]:
## writing dta to csv
author.to_csv('/Workspace/Users/ihekakelechi@outlook.com/dataset/transformeddata/author.csv', mode='w', index=False)
source.to_csv('/Workspace/Users/ihekakelechi@outlook.com/dataset/transformeddata/source.csv', mode='w', index=False)
article.to_csv('/Workspace/Users/ihekakelechi@outlook.com/dataset/transformeddata/article.csv', mode='w', index=False)
print('data written to csv correctly')

data written to csv correctly


In [ ]:
author_r.to_csv('/Workspace/Users/ihekakelechi@outlook.com/dataset/rawdata/author.csv', mode='w', index=False)
source_r.to_csv('/Workspace/Users/ihekakelechi@outlook.com/dataset/rawdata/source.csv', mode='w', index=False)
article_r.to_csv('/Workspace/Users/ihekakelechi@outlook.com/dataset/rawdata/article.csv', mode='w', index=False)
print('data written to csv correctly')

data written to csv correctly


### Loading Layer

In [ ]:
## setup connection 
# Azure Blob Storage connection string
connect_str = '**************************'
container_name = 'nunucontainer'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)


In [ ]:
def upload_df_to_blob_as_parquet(df, container_client, blob_name):
    buffer = io.BytesIO()
    df.to_parquet(buffer, index=False)
    buffer.seek(0)
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(buffer, blob_type="BlockBlob", overwrite=True)
    print(f'{blob_name} uploaded to Blob Storage successfully')

In [ ]:
upload_df_to_blob_as_parquet(author_r, container_client, 'rawdata/author.parquet')
upload_df_to_blob_as_parquet(source_r, container_client, 'rawdata/source.parquet')
upload_df_to_blob_as_parquet(article_r, container_client, 'rawdata/article.parquet')

rawdata/author.parquet uploaded to Blob Storage successfully
rawdata/source.parquet uploaded to Blob Storage successfully
rawdata/article.parquet uploaded to Blob Storage successfully


In [ ]:
upload_df_to_blob_as_parquet(author, container_client, 'transformeddata/author.parquet')
upload_df_to_blob_as_parquet(source, container_client, 'transformeddata/source.parquet')
upload_df_to_blob_as_parquet(article, container_client, 'transformeddata/article.parquet')

transformeddata/author.parquet uploaded to Blob Storage successfully
transformeddata/source.parquet uploaded to Blob Storage successfully
transformeddata/article.parquet uploaded to Blob Storage successfully
